In [1]:
getwd()

[1] "/Users/waynekao/CS424/bigYellowTaxi"

In [2]:
#READ CSV FILE AND CONVERT TO ONE DATA FRAME
col_name <- c("","Trip.Start.Timestamp","Trip.Seconds","Trip.Miles","Pickup.Community.Area","Dropoff.Community.Area","Company")
setwd("/Users/waynekao/CS424/bigYellowTaxi/Data/")
myfiles <- list.files(pattern="*.csv", full.names=TRUE)
myfiles
data <- do.call(rbind, lapply(myfiles, read.csv, header = FALSE))
colnames(data) <- col_name



[1] "./modified_taxi_partaa.csv" "./modified_taxi_partab.csv"
 [3] "./modified_taxi_partac.csv" "./modified_taxi_partad.csv"
 [5] "./modified_taxi_partae.csv" "./modified_taxi_partaf.csv"
 [7] "./modified_taxi_partag.csv" "./modified_taxi_partah.csv"
 [9] "./modified_taxi_partai.csv" "./modified_taxi_partaj.csv"
[11] "./modified_taxi_partak.csv" "./modified_taxi_partal.csv"
[13] "./modified_taxi_partam.csv" "./modified_taxi_partan.csv"
[15] "./modified_taxi_partao.csv" "./modified_taxi_partap.csv"
[17] "./modified_taxi_partaq.csv"

In [3]:
data

,Trip.Start.Timestamp,Trip.Seconds,Trip.Miles,Pickup.Community.Area,Dropoff.Community.Area,Company
19,01/01/2019 12:00:00 AM,1260,0.6,77,28,Blue Ribbon Taxi Association Inc.
21,01/01/2019 12:00:00 AM,360,0.8,8,8,Taxi Affiliation Services
23,01/01/2019 12:00:00 AM,360,1.0,8,8,Taxi Affiliation Services
24,01/01/2019 12:00:00 AM,420,0.8,8,8,"Taxicab Insurance Agency, LLC"
25,01/01/2019 12:00:00 AM,180,0.7,77,4,Taxi Affiliation Services
26,01/01/2019 12:00:00 AM,960,0.6,28,32,Taxi Affiliation Services
29,01/01/2019 12:00:00 AM,540,1.0,32,8,Choice Taxi Association
32,01/01/2019 12:00:00 AM,1800,3.2,32,8,Star North Management LLC
33,01/01/2019 12:00:00 AM,1560,14.3,76,8,Top Cab Affiliation
39,01/01/2019 12:00:00 AM,1980,5.8,32,32,Taxi Affiliation Services


In [4]:
library(lubridate)

n_date <- parse_date_time(data$Trip.Start.Timestamp,
                orders = 'mdY IMS %p', truncated = 3) #PARSE DATE FROM TIME STAMP
data$new_date <- n_date
data$Hour <- hour(data$new_date)
data


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



,Trip.Start.Timestamp,Trip.Seconds,Trip.Miles,Pickup.Community.Area,Dropoff.Community.Area,Company,new_date,Hour
19,01/01/2019 12:00:00 AM,1260,0.6,77,28,Blue Ribbon Taxi Association Inc.,2019-01-01,0
21,01/01/2019 12:00:00 AM,360,0.8,8,8,Taxi Affiliation Services,2019-01-01,0
23,01/01/2019 12:00:00 AM,360,1.0,8,8,Taxi Affiliation Services,2019-01-01,0
24,01/01/2019 12:00:00 AM,420,0.8,8,8,"Taxicab Insurance Agency, LLC",2019-01-01,0
25,01/01/2019 12:00:00 AM,180,0.7,77,4,Taxi Affiliation Services,2019-01-01,0
26,01/01/2019 12:00:00 AM,960,0.6,28,32,Taxi Affiliation Services,2019-01-01,0
29,01/01/2019 12:00:00 AM,540,1.0,32,8,Choice Taxi Association,2019-01-01,0
32,01/01/2019 12:00:00 AM,1800,3.2,32,8,Star North Management LLC,2019-01-01,0
33,01/01/2019 12:00:00 AM,1560,14.3,76,8,Top Cab Affiliation,2019-01-01,0
39,01/01/2019 12:00:00 AM,1980,5.8,32,32,Taxi Affiliation Services,2019-01-01,0


In [44]:
community_list <- c("Rogers Park", "West Ridge", "Uptown", 
          "Lincoln Square", "North Center", "Lake View",
         "Lincoln Park", "Near North Side", "Edison Park",
         "Norwood Park", "Jefferson Park", "Forest Glen",
         "North Park", "Albany Park", "Portage Park",
         "Irving Park", "Dunning", "Montclare", "Belmont Cragin",
         "Hermosa", "Avondale", "Logan Square", "Humboldt Park",
         "West Town", "Austin", "West Garfield Park",
         "East Garfield Park", "Near West Side", "North Lawndale",
         "South Lawndale", "Lower West Side", "Loop", "Near South Side",
         "Armour Square", "Douglas", "Oakland", "Fuller Park", 
         "Grand Boulevard", "Kenwood", "Washington Park", "Hyde Park",
         "Woodlawn", "South Shore", "Chatham", "Avalon Park", "South Chicago",
         "Burnside", "Calumet Heights", "Roseland", "Pullman", "South Deering",
         "East Side", "West Pullman", "Riverdale", "Hegewisch", "Garfield Ridge",
         "Archer Heights", "Brighton Park", "Mckinley Park", "Bridgeport",
         "New City", "West Elsdon", "Gage Park", "Clearing", "West Lawn",
         "Chicago Lawn", "West Englewood", "Englewood", "Greater Grand Crossing",
         "Ashburn", "Auburn Gresham", "Beverly", "Washington Heights",
         "Mount Greenwood", "Morgan Park", "Ohare", "Edgewater") # R IS INDEX STARTING 1 NOT 0

company_list <- c("1085 - 72312 N and W Cab Co", "1469 - 64126 Omar Jada","2092 - 61288 Sbeih company", "24 Seven Taxi",
                  "2733 - 74600 Benny Jona", "3011 - 66308 JBL Cab Inc.", "3094 - 24059 G.L.B. Cab Co", 
                  "312 Medallion Management Corp", "3556 - 36214 RC Andrews Cab", "3591 - 63480 Chuks Cab", 
                  "3620 - 52292 David K. Cab Corp.", "3623 - 72222 Arrington Enterprises", 
                  "3721 - Santamaria Express, Alvaro Santamaria", "4053 - 40193 Adwar H. Nikola", 
                  "4623 - 27290 Jay Kim", "5 Star Taxi", "5006 - 39261 Salifu Bawa", "5062 - 34841 Sam Mestas",
                  "5074 - 54002 Ahzmi Inc", "5874 - 73628 Sergey Cab Corp.", "6574 - Babylon Express Inc.",
                  "6742 - 83735 Tasha ride inc", "6743 - 78771 Luhak Corp", "American United", 
                  "American United Taxi Affiliation", "Blue Diamond", "Blue Ribbon Taxi Association Inc.",
                  "Checker Taxi", "Checker Taxi Affiliation", "Chicago Carriage Cab Corp", 
                  "Chicago Independents", "Chicago Medallion Management", "Chicago Star Taxicab", 
                  "Chicago Taxicab", "Choice Taxi Association", "City Service", "Flash Cab", "Globe Taxi", 
                  "Gold Coast Taxi", "KOAM Taxi Association", "Leonard Cab Co", "Medallion Leasin", 
                  "Metro Jet Taxi A", "Nova Taxi Affiliation Llc", "Patriot Taxi Dba Peace Taxi Associat", 
                  "Setare Inc", "Star North Management LLC", "Sun Taxi", "Taxi Affiliation Service Yellow", 
                  "Taxi Affiliation Services", "Taxicab Insurance Agency, LLC", "Top Cab Affiliation", "Yellow Cab",
                  "Petani Cab Corp", "U Taxicab")

#City of Chicago
community <- "All (City of Chicago)"
trip <- "To"
#All Company
company <- "All Taxis"
units <- "MI"

In [6]:
print(community_list[59]) #GRABBING COMMUNITY BY REGION ID
match("Dunning", community_list) #GETTING REGION ID BY COMMUNITY NAME


[1] "Mckinley Park"


[1] 17

In [7]:
sort(unique(data$Company))

[1] 1085 - 72312 N and W Cab Co                 
 [2] 1469 - 64126 Omar Jada                      
 [3] 2092 - 61288 Sbeih company                  
 [4] 24 Seven Taxi                               
 [5] 2733 - 74600 Benny Jona                     
 [6] 3011 - 66308 JBL Cab Inc.                   
 [7] 3094 - 24059 G.L.B. Cab Co                  
 [8] 312 Medallion Management Corp               
 [9] 3591 - 63480 Chuks Cab                      
[10] 3620 - 52292 David K. Cab Corp.             
[11] 3623 - 72222 Arrington Enterprises          
[12] 3721 - Santamaria Express, Alvaro Santamaria
[13] 4053 - 40193 Adwar H. Nikola                
[14] 4623 - 27290 Jay Kim                        
[15] 5 Star Taxi                                 
[16] 5006 - 39261 Salifu Bawa                    
[17] 5062 - 34841 Sam Mestas                     
[18] 5074 - 54002 Ahzmi Inc                      
[19] 5874 - 73628 Sergey Cab Corp.               
[20] 6574 - Babylon Express Inc.                 
[21] 6742 - 83735 Tasha ride inc                 
[22] 6743 - 78771 Luhak Corp                     
[23] American United                             
[24] American United Taxi Affiliation            
[25] Blue Diamond                                
[26] Blue Ribbon Taxi Association Inc.           
[27] Checker Taxi                                
[28] Checker Taxi Affiliation                    
[29] Chicago Carriage Cab Corp                   
[30] Chicago Independents                        
[31] Chicago Medallion Management                
[32] Chicago Star Taxicab                        
[33] Chicago Taxicab                             
[34] Choice Taxi Association                     
[35] City Service                                
[36] Flash Cab                                   
[37] Globe Taxi                                  
[38] Gold Coast Taxi                             
[39] KOAM Taxi Association                       
[40] Leonard Cab Co                              
[41] Medallion Leasin                            
[42] Metro Jet Taxi A                            
[43] Nova Taxi Affiliation Llc                   
[44] Patriot Taxi Dba Peace Taxi Associat        
[45] Setare Inc                                  
[46] Star North Management LLC                   
[47] Sun Taxi                                    
[48] Taxi Affiliation Service Yellow             
[49] Taxi Affiliation Services                   
[50] Taxicab Insurance Agency, LLC               
[51] Top Cab Affiliation                         
[52] Yellow Cab                                  
[53] Petani Cab Corp                             
[54] U Taxicab                                   
[55] 3556 - 36214 RC Andrews Cab                 
55 Levels: 1085 - 72312 N and W Cab Co ... 3556 - 36214 RC Andrews Cab

In [45]:
updateData <- data.frame(data)

if(community != "All (City of Chicago)"){
    if(trip == "To"){
        updateData <- subset(updateData, updateData$Dropoff.Community.Area == match(community, community_list))
    }
    else{ #FROM
        updateData <- subset(updateData, updateData$Pickup.Community.Area == match(community, community_list))
    }
}
if(company != "All Taxis"){
    updateData <- subset(updateData, updateData$Company == company)
}


In [46]:
updateData

Var.1,Trip.Start.Timestamp,Trip.Seconds,Trip.Miles,Pickup.Community.Area,Dropoff.Community.Area,Company,new_date,Hour
19,01/01/2019 12:00:00 AM,1260,0.6,77,28,Blue Ribbon Taxi Association Inc.,2019-01-01,0
21,01/01/2019 12:00:00 AM,360,0.8,8,8,Taxi Affiliation Services,2019-01-01,0
23,01/01/2019 12:00:00 AM,360,1.0,8,8,Taxi Affiliation Services,2019-01-01,0
24,01/01/2019 12:00:00 AM,420,0.8,8,8,"Taxicab Insurance Agency, LLC",2019-01-01,0
25,01/01/2019 12:00:00 AM,180,0.7,77,4,Taxi Affiliation Services,2019-01-01,0
26,01/01/2019 12:00:00 AM,960,0.6,28,32,Taxi Affiliation Services,2019-01-01,0
29,01/01/2019 12:00:00 AM,540,1.0,32,8,Choice Taxi Association,2019-01-01,0
32,01/01/2019 12:00:00 AM,1800,3.2,32,8,Star North Management LLC,2019-01-01,0
33,01/01/2019 12:00:00 AM,1560,14.3,76,8,Top Cab Affiliation,2019-01-01,0
39,01/01/2019 12:00:00 AM,1980,5.8,32,32,Taxi Affiliation Services,2019-01-01,0


In [35]:
#Aggregate by day
noFilter_byDay <- aggregate(updateData[,1], by=list(date(updateData$new_date)), FUN=length)
colnames(noFilter_byDay) <- c("Date", "Rides")
noFilter_byDay

Date,Rides
2019-01-01,8
2019-01-02,12
2019-01-03,14
2019-01-04,14
2019-01-05,5
2019-01-06,16
2019-01-07,9
2019-01-08,10
2019-01-09,14
2019-01-10,20


In [20]:
#Aggregate by hour
noFilter_byHour <- aggregate(updateData[,1], by=list(updateData$Hour), FUN=length)
colnames(noFilter_byHour) <- c("Hour", "Rides")
noFilter_byHour

Hour,Rides
0,247076
1,192008
2,147402
3,112595
4,87146
5,91697
6,159737
7,340740
8,533624
9,572724


In [21]:
#Aggregate by weekday
noFilter_byDayOfWeek <- aggregate(updateData[,1], by=list(wday(updateData$new_date, week_start=1)), FUN=length)
colnames(noFilter_byDayOfWeek) <- c("Day Of Week", "Rides")
#** STARTS MONDAY TO SUNDAY WITH 1 BEING MONDAY **
noFilter_byDayOfWeek

Day Of Week,Rides
1,1583114
2,1802296
3,1836651
4,1939243
5,1915833
6,1261132
7,1059737


In [64]:
#Aggregate by binned mileage 

#Add to global
mile_bin <- c("0.5 to < 10", "10 to < 20", "20 to < 30", "30 to < 40", "40 to < 50", "50 to < 60", "60 to < 70", "70 to < 80", "80 to < 90", "90 to 100")

#Actual code to get data frame
bin_data = data.frame(updateData)
bin_data$Mile_Bin <- NA
bin_data$Mile_Bin[bin_data$Trip.Miles >= 0.5 & bin_data$Trip.Miles < 10] <- mile_bin[1]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 10 & bin_data$Trip.Miles < 20] <- mile_bin[2]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 20 & bin_data$Trip.Miles < 30] <- mile_bin[3]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 30 & bin_data$Trip.Miles < 40] <- mile_bin[4]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 40 & bin_data$Trip.Miles < 50] <- mile_bin[5]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 50 & bin_data$Trip.Miles < 60] <- mile_bin[6]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 60 & bin_data$Trip.Miles < 70] <- mile_bin[7]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 70 & bin_data$Trip.Miles < 80] <- mile_bin[8]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 80 & bin_data$Trip.Miles < 90] <- mile_bin[9]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 90 & bin_data$Trip.Miles <= 100] <- mile_bin[10]
bin_data$Mile_Bin <- as.factor(bin_data$Mile_Bin)

noFilter_byBinnedMileage <- aggregate(bin_data[,1], by=list(bin_data$Mile_Bin), FUN=length)
colnames(noFilter_byBinnedMileage) <- c("Miles", "Rides")
noFilter_byBinnedMileage

Miles,Rides
0.5 to < 10,9640093
10 to < 20,1636894
20 to < 30,107931
30 to < 40,10991
40 to < 50,1385
50 to < 60,374
60 to < 70,152
70 to < 80,86
80 to < 90,52
90 to 100,48


In [68]:
#Aggregate by Trip Time
print(min(updateData$Trip.Seconds))
max(updateData$Trip.Seconds)

# #Add to global
time_bin <- c("1 minute to < 5 minutes", "5 minute to < 10 minutes", "10 minute to < 30 minutes", "30 minute to < 1 hr", "1 hr to < 2 hr", "2 hr to < 3 hr", "3 hr to < 4 hr", "4 hr to 5 hr")

# #Actual code to get data frame
bin_data = data.frame(updateData)
bin_data$Time_Bin <- NA
bin_data$Time_Bin[bin_data$Trip.Seconds >= 60 & bin_data$Trip.Seconds < 300] <- time_bin[1]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 300 & bin_data$Trip.Seconds < 600] <- time_bin[2]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 600 & bin_data$Trip.Seconds < 1800] <- time_bin[3]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 1800 & bin_data$Trip.Seconds < 3600] <- time_bin[4]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 3600 & bin_data$Trip.Seconds < 7200] <- time_bin[5]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 7200 & bin_data$Trip.Seconds < 10800] <- time_bin[6]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 10800 & bin_data$Trip.Seconds < 14400] <- time_bin[7]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 14400 & bin_data$Trip.Seconds < 18000] <- time_bin[8]
bin_data$Time_Bin <- factor(bin_data$Time_Bin, levels = time_bin, ordered = TRUE) 
noFilter_byBinnedTime <- aggregate(bin_data[,1], by=list(bin_data$Time_Bin), FUN=length)
colnames(noFilter_byBinnedTime) <- c("Time", "Rides")
noFilter_byBinnedTime



[1] 60


[1] 17999

Time,Rides
1 minute to < 5 minutes,1108056
5 minute to < 10 minutes,4116075
10 minute to < 30 minutes,4694215
30 minute to < 1 hr,1284155
1 hr to < 2 hr,187802
2 hr to < 3 hr,5211
3 hr to < 4 hr,1726
4 hr to 5 hr,766


In [69]:
summary(noFilter_byBinnedTime)

                        Time       Rides        
 1 minute to < 5 minutes  :1   Min.   :    766  
 5 minute to < 10 minutes :1   1st Qu.:   4340  
 10 minute to < 30 minutes:1   Median : 647929  
 30 minute to < 1 hr      :1   Mean   :1424751  
 1 hr to < 2 hr           :1   3rd Qu.:1992135  
 2 hr to < 3 hr           :1   Max.   :4694215  
 (Other)                  :2                    